In [ ]:
import yaml

path_to_yaml = "config/config.yaml"

with open(path_to_yaml, "r") as yaml_file:
    content = yaml.safe_load(yaml_file)
    print("YAML Content:", content)  # Should print the dictionary


YAML Content: {'artifacts_root': 'artifacts', 'data_ingestion': {'root_dir': 'artifacts/data_ingestion', 'source_URL': 'https://drive.google.com/file/d/1ckBFV4rP4tiCP5Q8xTpZTH2naliiU01m/view?usp=sharing', 'local_data_file': 'artifacts/data_ingestion/data.zip', 'unzip_dir': 'artifacts/data_ingestion'}}


In [52]:

import os

In [53]:
%pwd


'C:\\Users\\samri\\OneDrive\\Documents\\Desktop\\Kidney-disesase-classification'

In [54]:

os.chdir("../")

In [55]:
%pwd

'C:\\Users\\samri\\OneDrive\\Documents\\Desktop'

In [56]:
%cd c:\\Users\\samri\\OneDrive\\Documents\\Desktop\\Kidney-disesase-classification'

[WinError 2] The system cannot find the file specified: "c:\\\\Users\\\\samri\\\\OneDrive\\\\Documents\\\\Desktop\\\\Kidney-disesase-classification'"
C:\Users\samri\OneDrive\Documents\Desktop\Kidney-disesase-classification


In [57]:
%pwd

'C:\\Users\\samri\\OneDrive\\Documents\\Desktop'

In [58]:
%cd C:\Users\samri\OneDrive\Documents\Desktop\Kidney-disesase-classification

C:\Users\samri\OneDrive\Documents\Desktop\Kidney-disesase-classification


In [59]:
%pwd

'C:\\Users\\samri\\OneDrive\\Documents\\Desktop\\Kidney-disesase-classification'

In [60]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [61]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories


In [62]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )
        return data_ingestion_config


In [63]:
import os
import zipfile
import gdown
from cnnClassifier import logger
from cnnClassifier.utils.common import get_size

In [ ]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    
    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    

    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [66]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2024-11-26 17:16:02,477:INFO:common:yaml file:config\config.yaml loaded sucessfully]
[2024-11-26 17:16:02,477:INFO:common:yaml file:params.yaml loaded sucessfully]
[2024-11-26 17:16:02,483:INFO:common:Created directory at :artifacts]
[2024-11-26 17:16:02,485:INFO:common:Created directory at :artifacts/data_ingestion]
[2024-11-26 17:16:02,487:INFO:986331348:Downloading data from https://drive.google.com/file/d/1CkBfV4rP4tiCP5Q8XTpZTH2naliiuO1m/view?usp=sharing into file artifacts/data_ingestion/data.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1CkBfV4rP4tiCP5Q8XTpZTH2naliiuO1m
From (redirected): https://drive.google.com/uc?%2Fexport=download&id=1CkBfV4rP4tiCP5Q8XTpZTH2naliiuO1m&confirm=t&uuid=63cc09d1-46a4-4c94-8aab-2837d5ecf04c
To: C:\Users\samri\OneDrive\Documents\Desktop\Kidney-disesase-classification\artifacts\data_ingestion\data.zip
100%|██████████| 941M/941M [12:47<00:00, 1.23MB/s]   

[2024-11-26 17:30:06,209:INFO:986331348:Downloaded data from https://drive.google.com/file/d/1CkBfV4rP4tiCP5Q8XTpZTH2naliiuO1m/view?usp=sharing into file artifacts/data_ingestion/data.zip]
